In [25]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-1
    # Size x
    _, dim = x.shape
    # Gradient vector
    if multioutput:
        gradient = np.zeros((dim, dim))
    else:
        gradient = np.zeros((dim, 1))
    for i in range(dim):
        # Vector for partial derivative estimation
        offset_tensor = np.zeros((1, dim))
        offset_tensor[0, i] = h
        offset_tensor = tf.convert_to_tensor(offset_tensor,
                                             dtype = tf.float32)
        # Constantes:
        denominator = h
        if mode == 'progressive':
            numerator = model(
                tf.math.add(x, offset_tensor)
            ) - model(
                x
            ) 
        elif mode == 'regressive':
            numerator = model(
                x
            ) - model(
                tf.math.subtract(x, offset_tensor)
            )
        elif mode == 'centered':
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
        denominator = 2 * h
        gradient[i, :] = numerator / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [142]:

input_layer = keras.Input(shape = (10,), name='input_nn')
'''gru_unit = layers.GRU(
    128, 
    name = 'sequential_layer'
)(input_layer)'''
dense_unit = layers.Dense(
    units = 64,
    name = 'dense_layer'
)(input_layer)
output_layer = layers.Dense(
    units = 10, 
    activation = 'relu', 
    name = 'first_dense'
)(dense_unit)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

In [179]:
# Domain -5:5
x = tf.convert_to_tensor(
    np.array(range(-5,5)).reshape((1,10)),
    dtype = tf.float32    
)
# Test 
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(xs)
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})

In [180]:
custom_grads = tf.transpose(custom_derivative_model(x, custom_model))

In [172]:
x

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.]], dtype=float32)>

In [181]:
grads['x'], custom_grads, jacobian['x']

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-0.07849032, -0.8184525 , -0.4894024 ,  0.31033385, -0.01098841,
          0.33701032,  0.01973917,  0.27462688, -0.05480143,  0.5937259 ]],
       dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[-0.14227033,  0.        ,  0.        ,  0.12732685,  0.05387843,
          0.        , -0.11742949,  0.        ,  0.        ,  0.        ],
        [-0.13334334,  0.        ,  0.        , -0.22000134, -0.17940164,
          0.        , -0.28570652,  0.        ,  0.        ,  0.        ],
        [ 0.09474874,  0.        ,  0.        , -0.12032986, -0.0159359 ,
          0.        , -0.44789553,  0.        ,  0.        ,  0.        ],
        [ 0.00095427,  0.        ,  0.        ,  0.12259454,  0.1000917 ,
          0.        ,  0.086689  ,  0.        ,  0.        ,  0.        ],
        [-0.05466878,  0.        ,  0.        , -0.10521203, -0.00908315,
          0.        ,  0.15797257,  0.        ,  0.    